In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics

In [2]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')
train_support = pd.read_csv('../train.csv')
test_support = pd.read_csv('../test.csv')
#df_train['WnvPresent'] = train_support['WnvPresent']
#df_train.Date = train_support.Date
#df_test.Date = test_support.Date
df_trainSpecies = train_support.Species
df_test.Species = test_support.Species

In [3]:
df_train['counter'] = 1

In [4]:
X = df_train.copy()


In [5]:
string_features = ['Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
#trap_encoder = preprocessing.LabelEncoder()
X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
#X.Species = species_encoder.fit_transform(X.Species)
#y = df_train['NumMosquitos']
#df_train.Species = species_encoder.transform(df_train.Species)
X = X.drop(string_features,axis=1)

/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
groups = ['Date','Trap','Species']

In [7]:
tmp = X.groupby(groups)['NumMosquitos'].sum()


In [8]:
X_tmp = X.join(X.groupby(groups)['NumMosquitos'].sum(), on= groups,rsuffix='_counter')

In [9]:
X_tmp.NumMosquitos_counter.max()

2532

In [10]:
y = X_tmp.NumMosquitos_counter


In [11]:
#X_tmp.drop('Date',axis=1)
#X_tmp.drop('Trap',axis = 1)#del X.Trap
del X_tmp['Trap']
del X_tmp['Date']
del X_tmp['NumMosquitos']
del X_tmp['NumMosquitos_counter']
del X_tmp['WnvPresent']
del X_tmp['counter']
del X_tmp['Unnamed: 0']
#['Unnamed: 0', 'WnvPresent', 'counter']

X_tmp.Species = species_encoder.fit_transform(X.Species)
df_test.Species = species_encoder.transform(df_test.Species)
df_train.species = species_encoder.transform(df_train.Species)

In [12]:
X = X_tmp.copy()

In [13]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, SGDRegressor
import numpy as np

In [14]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [15]:

clf_list= [#RandomForestRegressor(n_estimators=100,n_jobs=-1),
           #RandomForestRegressor(n_estimators=400,n_jobs=-1),
           #RandomForestRegressor(n_estimators=1000,n_jobs=-1), 
           #Ridge(alpha = .01,normalize=True),
           #Ridge(alpha = .1,normalize=True),
           #Ridge(alpha = .1),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=7),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=10),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=15,subsample = .7),#so far the best one, n_es = 400
           GradientBoostingRegressor(n_estimators = 4000,max_depth=15,learning_rate = 0.01),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=20,subsample = .7)
           #KNeighborsRegressor(),
           #SGDRegressor()
           #GradientBoostingRegressor(n_estimators = 100,max_depth=4),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=2)
      ]
random_state= 46


In [16]:
X.dtypes

Species                      int64
Block                        int64
Latitude                   float64
Longitude                  float64
AddressAccuracy              int64
Calendar_Week                int64
Station                      int64
Tmax                         int64
Tmin                         int64
Tavg                       float64
Depart                     float64
DewPoint                     int64
WetBulb                    float64
Heat                         int64
Cool                         int64
Sunrise                    float64
Sunset                     float64
Depth                      float64
SnowFall                   float64
PrecipTotal                float64
StnPressure                float64
SeaLevel                   float64
ResultSpeed                float64
ResultDir                    int64
AvgSpeed                   float64
Year                         int64
Month                        int64
Tavg_week                  float64
precip_week         

In [ ]:
X = scale(X)


err = []
r2 = []
#get lengths of y_arr -.-

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)

y_arr = pd.DataFrame(y_test)
i = 1
#clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    

In [ ]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

In [ ]:
r2

In [ ]:
X_tmp

In [ ]:
df_train['count_mosquitos'] = y

In [ ]:
clf.fit(X,y)

In [ ]:
df_test = df_test.drop(string_features,axis=1)


In [ ]:
[var for var in df_test.columns if var not in X_tmp.columns]

In [ ]:
[var for var in X_tmp.columns if var not in df_test.columns]

In [ ]:
df_test = df_test.drop(['Id', 'Date', 'Trap'],axis=1)

In [ ]:
[var for var in df_test.columns if var not in X_tmp.columns]

In [ ]:
[var for var in X_tmp.columns if var not in df_test.columns]

In [ ]:
[var for var in X_tmp.columns if var not in df_train.columns]

In [ ]:
[var for var in df_train.columns if var not in X_tmp.columns]

In [ ]:
df_test2 = scale(df_test)

In [ ]:
yhat = clf.predict(df_test2)

In [ ]:
yhat.mean(),yhat.std(),y.mean(),y.std()

In [ ]:
yhat

In [ ]:
df_test['count_mosquitos'] = yhat

In [ ]:
[var for var in df_train if var not in df_test]

In [ ]:
strf = ['Unnamed: 0',
 'Date',
 'Address',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'NumMosquitos',
 #'WnvPresent',
 'CodeSum',
 'counter']

In [ ]:
df_train = df_train.drop(strf,axis=1)

In [ ]:
[var for var in df_train if var not in df_test]

In [ ]:
[var for var in df_test if var not in df_train]

In [ ]:
train_support = pd.read_csv('../train.csv')
test_support = pd.read_csv('../test.csv')
df_train['WnvPresent'] = train_support['WnvPresent']
df_train.Date = train_support.Date
df_test.Date = test_support.Date
df_trainSpecies = train_support.Species
df_test.Species = test_support.Species

In [ ]:
df_test.to_csv('new_test.csv',index_label='Id')
df_train.to_csv('new_train.csv',index_label='Id')

In [ ]:
df_train.WnvPresent